In [1]:
import pandas as pd
import numpy as np

In [42]:
df_final = pd.read_csv(r'/home/refu0917/lungcancer/server/AllCaseCtrl_final.csv')
df_raw = pd.read_csv(r'/home/refu0917/lungcancer/server/AllCaseCtrl_RAW.csv')
col_raw = ["LOC", "IDCODE", "CNRSEQ", "Class", "Gender", "Age", "DIFF", "TMRSZ", "LYMND"
            , "AJCCstage", "BMI_label", "MAGN", "RTDATE", "STDATE", "DEFISURPRIDAT" , "CIG"
            , "BN", "ALC", "SSF1", "SSF2", "SSF3", "SSF4", "SSF6", "FullDate"    
            ]
col_final = ["LOC", "IDCODE", "CNRSEQ", "Class", "Gender", "Age", "DIFF", "TMRSZ", "LYMND"
            , "AJCCstage", "BMI_label", "MAGN", "RTDATE", "STDATE", "OP" , "CIG"
            , "BN", "ALC", "SSF1", "SSF2", "SSF3", "SSF4", "SSF6", "FullDate"    
            ]
df_raw = df_raw.loc[(df_raw['CLASSOFCASE'] == 1) | (df_raw['CLASSOFCASE'] == 2)]
df_raw = df_raw[col_raw]
df_final = df_final[col_final]
df_raw['Class'] = df_raw['Class'].apply(lambda x:0 if x == 0 else 1)
df_final['Class'] = df_final['Class'].apply(lambda x:0 if x == 0 else 1)

In [24]:
df_raw['DEFISURPRIDAT'] = df_raw['DEFISURPRIDAT'].fillna(1)
df_raw['OP'] = df_raw['DEFISURPRIDAT'].apply(lambda x:2 if x != 1 else 1)
df_raw = df_raw.drop(columns=['DEFISURPRIDAT'])

In [25]:
df_final['year'] = df_final['FullDate'].apply(lambda x : int(x[:4]))
df_raw['year'] = df_raw['FullDate'].apply(lambda x : int(x[:4]))
# df_final = df_final.drop(columns = ['FullDate'])
# df_raw = df_raw.drop(columns = ['FullDate'])

In [34]:
df_raw = df_raw.loc[(df_raw.year >= 2011) & (df_raw.CNRSEQ == 1)]
df_final = df_final.loc[(df_final.year >= 2011) & (df_final.CNRSEQ == 1)]

In [38]:
df_final.isna().sum(axis=1).value_counts()

0    6562
3    1270
1     419
2      37
4      11
9       1
dtype: int64

In [39]:
df_raw.isna().sum(axis=1).value_counts()

0    6553
1     419
2      35
3       9
9       1
dtype: int64

In [41]:
df_raw.to_csv(r'/home/refu0917/lungcancer/server/AllCaseCtrl_RAW_Process.csv')

In [40]:
df_raw = pd.read_csv(r'/home/refu0917/lungcancer/server/AllCaseCtrl_RAW_Process.csv',index_col=[0])
df_final = pd.read_csv('/home/refu0917/lungcancer/server/AllCaseCtrl_final.csv')
columns = ["Class","LOC", "FullDate","Gender", "Age", "CIG",
            "ALC", "BN", "MAGN", "AJCCstage", "DIFF", "LYMND",
            "TMRSZ", "OP", "RTDATE", "STDATE", "BMI_label",
            "SSF1", "SSF2", "SSF3", "SSF4", "SSF6"]
df_raw = df_raw[columns]
df_final = df_final[columns]

In [63]:
df_final[columns].isna().sum(axis = 1).value_counts()


0    6562
3    1270
1     419
2      37
4      11
9       1
dtype: int64

In [43]:
df_final['year'] = df_final['FullDate'].apply(lambda x:int(x[:4]))
df_final = df_final[df_final['year']>=2011]
df_final.isna().sum(axis=1).value_counts()
df_raw['null_count'] = df_raw.isna().sum(axis=1)

In [14]:
fpc = pd.read_csv('/home/refu0917/lungcancer/server/fp_cancer.csv')
spc = pd.read_csv('/home/refu0917/lungcancer/server/sp_cancer.csv')
columns = ['醫院別', '歸戶代號', '有無二癌', '性別', '診斷年齡', '分級分化','病理分級分化','臨床分級分化', '腫瘤大小', '區域淋巴結侵犯數目', '臨床期別組合'
        , '原發部位手術邊緣', '放射治療開始日期', '全身性治療開始日期', '原發部位最確切的手術切除日期', '身高'
        , '體重', '吸菸行為' ,'嚼檳榔行為', '喝酒行為', '癌症部位特定因子1', '癌症部位特定因子2'
        , '癌症部位特定因子3', '癌症部位特定因子4', '癌症部位特定因子6', '最初診斷日期', '個案分類', '序']

col_map = ["LOC", "id","Class", "Gender", "Age", "DIFF","DIFF_path","DIFF_clin", "TMRSZ", "LYMND"
            , "AJCCstage", "MAGN", "RTDATE", "STDATE", "OP", "Height", "Weight"
            , "CIG", "BN", "ALC", "SSF1", "SSF2", "SSF3", "SSF4", "SSF6", "FullDate", 'Classofcase', 'Seq'    
            ]
fpc = fpc[columns]
spc = spc[columns]
fpc = fpc.rename(columns=dict((i,j)for i,j in zip(columns, col_map)))
spc = spc.rename(columns=dict((i,j)for i,j in zip(columns, col_map)))

spc = spc[spc.Class == 'Y']
df_all = pd.concat([fpc, spc])
df_all = df_all[df_all.LOC != '長庚']
df_all['DIFF_path'] = df_all['DIFF_path'].replace('D', 9)


/tmp/ipykernel_336127/4152459814.py:1: DtypeWarning: Columns (21,120) have mixed types. Specify dtype option on import or set low_memory=False.
  fpc = pd.read_csv('/home/refu0917/lungcancer/server/fp_cancer.csv')


In [15]:
temp = df_all[df_all.DIFF.isna()]
temp['DIFF_path'] = temp['DIFF_path'].astype(int)
temp['DIFF_clin'] = temp['DIFF_clin'].astype(int)
temp['DIFF'] = temp.apply(lambda x:x['DIFF_path'] if x['DIFF_path'] < x['DIFF_clin'] else x['DIFF_clin'], axis=1)
df_all.loc[df_all.DIFF.isna(), 'DIFF'] = temp['DIFF']
df_all = df_all.drop(columns = ['DIFF_path', 'DIFF_clin'])

/tmp/ipykernel_336127/4016117515.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp['DIFF_path'] = temp['DIFF_path'].astype(int)
/tmp/ipykernel_336127/4016117515.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp['DIFF_clin'] = temp['DIFF_clin'].astype(int)
/tmp/ipykernel_336127/4016117515.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.p

In [25]:
pd.set_option('display.max_columns', None)
aaa = pd.read_csv('/home/refu0917/lungcancer/server/5site_encode.csv')
aaa[aaa['醫院別'] != '長庚']

,有無二癌,性別,診斷年齡,側性,組織類型,分級分化,腫瘤大小,區域淋巴結侵犯數目,整併期別,原發部位手術邊緣,放射治療,體外放射治療技術,放射治療儀器,放射治療與手術順序,放射治療臨床標靶體積摘要,全身性治療,首次復發型式,BMI,吸菸行為,嚼檳榔行為,喝酒行為,SSF1_同側肺但非主病灶的另外肺腫瘤,SSF2_波及臟層膜 彈性層,SSF3_首次治療前生活功能狀態的評估,SSF4_惡性肋膜積水,SSF6_EGFR 基因突變,SSF7_ALK 基因轉位突變,原發部位最確切的手術切除日期,醫院別
9295,1,2,6,2,2,9,1,1,1,1,1,0,0,0,9,1,1,3,1,1,1,1,1,9,9,9,9,20180413,中山
9296,1,1,4,2,5,2,1,1,2,1,1,0,0,0,9,1,1,3,1,1,1,1,1,9,9,9,9,20200108,中山
9297,1,2,6,2,5,2,1,1,2,1,1,0,0,0,9,2,1,3,1,1,1,1,2,1,9,2,9,20160601,中山
9298,1,1,4,2,4,3,1,9,4,9,2,4,1,9,3,2,1,2,2,1,1,1,9,1,9,9,9,0,中山
9299,1,2,5,3,2,9,1,9,5,9,1,0,0,9,9,2,3,2,1,1,1,2,9,9,1,2,9,0,中山
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14058,2,1,5,2,2,3,1,3,5,1,2,2,1,9,4,2,1,2,2,2,1,1,2,2,1,1,1,20170418,亞東
14059,2,1,6,3,2,3,2,9,5,9,1,0,0,9,9,2,3,3,2,1,2,1,9,2,2,2,1,0,亞東
14060,2,2,5,3,2,9,9,1,2,9,1,0,0,0,9,2,1,9,1,9,9,1,1,9,9,9,9,20130699,仁愛
14061,2,2,4,3,5,2,1,1,2,1,1,0,0,0,9,1,1,2,1,1,1,1,1,9,9,9,9,20190628,仁愛


In [4]:
def age_encode(age):
    if age < 21:
        return 1
    elif 21 <= age < 31:
        return 2
    elif 31 <= age < 41:
        return 3
    elif 41 <= age < 51:
        return 4
    elif 51 <= age < 61:
        return 5
    elif 61 <= age < 71:
        return 6
    elif 71 <= age < 81:
        return 7
    elif 81 <= age < 91:
        return 8
    elif age >= 91:
        return 9
def size_encode(size):
    if 1<=size<=49 or 991 <= size <= 995:
        return 1
    elif 50 <= size <= 99:
        return 2
    elif 100 <= size <= 149:
        return 3
    elif 150 <= size:
        return 4
    elif size in [990,998,999]:
        return 9
def lymnd_encode(x):
    if x == 0:
        return 1
    elif 1 <= x <= 2:
        return 2
    elif 3 <= x <= 6:
        return 3
    elif 7 <= x <= 15:
        return 4
    elif 16 <= x:
        return 5
    elif x in [95,97,98,99]:
        return 9
def ajcc_encode(x):
    
    if x in ['BBB', '888', '999', 'OC ']:
        return 9
    else:
        temp = int(x[0])
        if temp == 0:
            return 1
        elif temp == 1:
            return 2
        elif temp == 2:
            return 3
        elif temp == 3:
            return 4
        elif temp == 4:
            return 5

def magn_encode(x):
    if x == 0:
        return 1
    elif x in[7,8,9]:
        return 9
    else:
        return 2
def date_encode(date):
    if date == 0:
        return 0
    else: 
        return 1

def cig_encode(cig):
    if cig == 88:
        return 1
    elif cig == 999999:
        return 9
    else:
        return 2
def alc_encode(alc):
    if alc == 0:
        return 1
    elif alc == 999:
        return 9
    else: return 2
def bn_encode(bn):
    if bn == 88:
        return 1
    elif bn == 999:
        return 9
    else: return 2
def ssf1_encode(x):
    if x == 0 or x == '000':
        return 1
    elif x in[10,20,30,40,'030','010','020','040']:
        return 2
    elif x in [999, 988,'988','999']:
        return 9
def ssf2_encode(x):
    if x == 0 or x == '000':
        return 1
    elif x in[10,20,30,40,'010','020','030','040']:
        return 2
    elif x in [999, 988, '988', '999']:
        return 9
    else: return x
def ssf3_encode(x):
    if x in [999,988,998]:
        return 9
    else: return x+1 
def ssf4_encode(x):
    if x == 0:
        return 1
    elif 11<=x<=15:
        return 2
    elif x in [999,988]:
        return 9
def ssf6_encode(x):
    if x[0] == '0' or x[0] == '9':
        if x[1] == '2':
            return 1
        elif x[1] == '1':
            return 2
        else: return 9
    else:
        if x[0]  == 'X':
            return 1
        elif x[0] == 'Z':
            return 9
        else: return 2

In [5]:
df_all['Class'] = df_all['Class'].apply(lambda x:0 if x == 'N' else 1)
df_all['Age'] = df_all['Age'].apply(lambda x:age_encode(x))
df_all['TMRSZ'] = df_all['TMRSZ'].apply(lambda x:size_encode(x))
df_all['LYMND'] = df_all['LYMND'].apply(lambda x:lymnd_encode(x))
df_all['AJCCstage'] = df_all['AJCCstage'].apply(lambda x:ajcc_encode(x))
df_all['MAGN'] = df_all['MAGN'].apply(lambda x:magn_encode(x))
df_all['RTDATE'] = df_all['RTDATE'].apply(lambda x:date_encode(x))
df_all['STDATE'] = df_all['STDATE'].apply(lambda x:date_encode(x))
df_all['OP'] = df_all['OP'].apply(lambda x:date_encode(x))
df_all['Height'] = df_all['Height'].apply(lambda x:(x/100)**2)
df_all['BMI'] = df_all['Weight']/df_all['Height']
df_all['CIG'] = df_all['CIG'].apply(lambda x:cig_encode(x))
df_all['BN'] = df_all['BN'].apply(lambda x:bn_encode(x))
df_all['ALC'] = df_all['ALC'].apply(lambda x:alc_encode(x))
df_all['SSF1'] = df_all['SSF1'].apply(lambda x:ssf1_encode(x))
df_all['SSF2'] = df_all['SSF2'].apply(lambda x:ssf2_encode(x))
df_all['SSF3'] = df_all['SSF3'].apply(lambda x:ssf3_encode(x))
df_all['SSF4'] = df_all['SSF4'].apply(lambda x:ssf4_encode(x))
df_all['SSF6'] = df_all['SSF6'].apply(lambda x:ssf6_encode(x))

In [12]:
df_all[df_all.Height>190]

,LOC,id,Class,Gender,Age,DIFF,TMRSZ,LYMND,AJCCstage,MAGN,...,ALC,SSF1,SSF2,SSF3,SSF4,SSF6,FullDate,Classofcase,Seq,BMI


In [8]:
df_all

,LOC,id,Class,Gender,Age,DIFF,TMRSZ,LYMND,AJCCstage,MAGN,...,ALC,SSF1,SSF2,SSF3,SSF4,SSF6,FullDate,Classofcase,Seq,BMI
9295,中山,1002694,0,2,6,9,1,1,1,2,...,1,1,1.0,9.0,9,9,20180410,1,1,24.557752
9296,中山,1004902,0,1,4,2,1,1,2,2,...,1,1,1.0,9.0,9,9,20191108,2,1,30.795848
9297,中山,1005641,0,2,6,2.0,1,1,2,2,...,1,1,2.0,1.0,9,2,20160510,1,1,26.402355
9298,中山,1010218,0,1,4,3.0,1,5,4,2,...,1,1,9.0,1.0,9,9,20170801,1,1,23.738662
9299,中山,1013166,0,2,5,9,1,5,5,2,...,1,2,9.0,9.0,1,2,20180831,2,1,20.342798
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1122,亞東,W96928,1,1,5,3,1,3,5,2,...,1,1,2.0,2.0,1,1,20170399,2,1,21.612812
1124,亞東,Y02737,1,1,6,3,2,5,5,2,...,2,1,9.0,2.0,2,2,20170411,1,1,24.464602
1126,仁愛,00264500,1,2,5,9,4,1,9,2,...,9,1,1.0,9.0,9,9,20130426,2,1,10.010010
1128,仁愛,00629291,1,2,4,2,1,1,2,2,...,1,1,1.0,9.0,9,9,20190603,1,1,23.634033
